In [1]:
import pandas as pd

In [4]:
data = pd.read_csv('stackoverflow_reduced.csv')


In [5]:
data.head()

,post,tags
0,identifying server timeout quickly in iphone ...,iphone
1,distance between 2 or more drop pins i was do...,iphone
2,what are all the restrictions by apple for iph...,iphone
3,not able to clicked on item i have facing ver...,iphone
4,displaying addressbook data acording to a cert...,iphone


In [6]:
print("Input data has {} rows and {} columns".format(len(data), len(data.columns)))

Input data has 3000 rows and 2 columns


In [16]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string
from bs4 import BeautifulSoup

stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

In [17]:
def clean_text(text):
    text = BeautifulSoup(text, "lxml").text
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    text = BAD_SYMBOLS_RE.sub('', text) 
    tokens = re.split('\W+', text)
    text = [wn.lemmatize(word) for word in tokens if word not in stopwords]
    return text

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['post'], data['tags'], test_size=0.2)

In [28]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text, min_df=2, encoding='latin-1', ngram_range=(1, 2))
tfidf_vect_fit = tfidf_vect.fit(X_train)

tfidf_train = tfidf_vect_fit.transform(X_train)
tfidf_test = tfidf_vect_fit.transform(X_test)

X_train_vect = pd.DataFrame(tfidf_train.toarray())
X_test_vect = pd.DataFrame(tfidf_test.toarray())
X_train_vect.head()

,0,1,2,3,4,5,6,7,8,9,...,5438,5439,5440,5441,5442,5443,5444,5445,5446,5447
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.078425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.267331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [30]:
import xgboost as xgb


In [31]:
xgb = xgb.XGBClassifier(objective="multi:softprob", random_state=42, n_estimators=150, max_depth=11)

xgb_model = xgb.fit(X_train_vect, y_train)
y_pred = xgb_model.predict(X_test_vect)

precision, recall, fscore, train_support = score(y_test, y_pred)
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
accuracy = ((y_pred==y_test).sum()/len(y_pred))
print('Accuracy: {}'.format(accuracy))
    

C:\Users\Swetha\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Precision: [0.95067265 0.97326203 0.98421053]
Recall: [0.96803653 0.95789474 0.97905759]
Accuracy: 0.9683333333333334
